## Data Analysis
This notebook will be used to analyse data from Combination Lock experiments.

### Install Libraries

In [78]:
%%capture

%pip install matplotlib
import json
import glob

### Read in Files

In [79]:
log_dir = './docs/evaluation-logs/*'
filepaths = glob.glob(log_dir)

files = []

for path in filepaths:
    with open(path) as f:
        files.append(json.load(f)['data'])

### Extract Data Across Participants into Single Dict

In [80]:
fullDict = {}


for f in files:
    fullDict = fullDict | f

print(len(fullDict))

36


### Traverse entire dataset, adding each list to the appropraite method list

In [82]:
methodsDict = {'tap6':[], 'tap10':[], 'direction6':[], 'direction10':[], 'pin':[],'pattern':[]}

for key in fullDict.keys():
    method = fullDict[key][0]['method']
    numTicks = fullDict[key][0]['numTicks']

    if method == 'tap':
        if numTicks == '6':
            methodsDict['tap6'] += fullDict[key]
        else:
            methodsDict['tap10'] += fullDict[key]
    elif  method == 'direction':
        if numTicks == '6':
            methodsDict['direction6'] += fullDict[key]
        else:
            methodsDict['direction10'] += fullDict[key]
    elif method == 'PATTERN':
        methodsDict['pattern'] += fullDict[key]
    elif method == 'PIN':
        methodsDict['pin'] += fullDict[key]
    
# Remove Abortions
for key in methodsDict:
    methodsDict[key] = list(filter(lambda x:x['endState'] != 'ABORTED', methodsDict[key]))

### Graph Average Success Rate For Each Method

In [89]:
successRates = {}


def computeErrorRate(lst):
    errors = len(list(filter(lambda x: x['endState'] == 'FAILURE', lst)))
    return (errors / len(lst)) * 100

for method in methodsDict:
    successRates[method] = computeErrorRate(methodsDict[method])

{'tap6': 1.8181818181818181, 'tap10': 4.424778761061947, 'direction6': 31.645569620253166, 'direction10': 32.5, 'pin': 5.263157894736842, 'pattern': 12.195121951219512}
